In [1]:
import numpy as np
import pandas as pd

# conda install nltk 
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics.scorer import SCORERS

In [2]:
# import nltk
# nltk.download()

In [3]:
try:
    movie_reviews.categories()
except:
    import nltk
    print('This appears to be your first time using the NLTK Movie Reviews corpus. We will first download the necessary corpus (this is a one-time download that might take a little while')
    nltk.download('movie_reviews')
#     from nltk.corpus import movie_reviews

In [19]:
def write_answer(num, answer):
    with open("dminik6_p4_w1_{}.txt".format(num), "w") as fout:
        fout.write(str(answer))

1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [4]:
# Чтобы получить id-шники негативных и позитивных отзывов:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [5]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

http://www.nltk.org/book/ch06.html#document-classification

2. Подсчитайте количество отзывов в выборке.

In [146]:
ans_1 = len(negids) + len(posids)
print('1. Количество отзывов в выборке:{}'.format(ans_1))
write_answer(1, ans_1)

1. Количество отзывов в выборке:2000


In [15]:
# print(texts[10])

3. Подсчитайте долю класса 1 в выборке.

In [145]:
ans_2 = len(posfeats) / (ans_1)
print('2. Доля класса 1 в выборке:{}'.format(ans_2))
write_answer(2, ans_2)

2. Доля класса 1 в выборке:0.5


4. Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [147]:
cv = CountVectorizer()
X = cv.fit_transform(texts)
ans_3 = X.shape[1]
print('3. Количество признаков в CountVectorizer со стандартными настройками: {}'.format(ans_3))
print(len(cv.get_feature_names()))
write_answer(3, ans_3)

3. Количество признаков в CountVectorizer со стандартными настройками: 39659
39659


5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [9]:
# Прежде чем начать работать с качеством, посмотрим какие "оенщики" есть 
print(SCORERS.keys())

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])


In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('logreg', LogisticRegression(solver = 'liblinear'))
])

# По умолчанию scoring устанавливаетмя в значение accuracy. Но можно и написать явно 
accuracy = cross_val_score(pipeline, texts, labels, cv=3, scoring='accuracy' )



In [144]:
print(accuracy)
ans_4 = accuracy.mean()
print('4. Accuracy в кросс-валидации: {}'.format(ans_4))
write_answer(4, ans_4)

[0.81437126 0.84684685 0.84684685]
4. Accuracy в кросс-валидации: 0.8360216503929078


6. Аналогично accuracy, оцените качество по ROC AUC.

In [12]:
roc_auc = cross_val_score(pipeline, texts, labels, scoring='roc_auc', cv=3 )

In [143]:
print(roc_auc)
ans_5 = roc_auc.mean()
print('5. ROC AUC в кросс-валидации: {}'.format(ans_5))
write_answer(5, ans_5)

[0.9006239  0.91283175 0.91887383]
5. ROC AUC в кросс-валидации: 0.9107764937833774


7. Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

Запишите соответствующие двум самым важным для модели признакам слова из словаря через пробел

In [62]:
lr = LogisticRegression(solver = 'liblinear').fit(X, labels)

In [142]:
# nword = 2
# ans_6 = ' '.join([cv.get_feature_names()[i] for i in np.argpartition(-lr.coef_[0], nword)[:nword]])
# print('Два самых важным для модели признакам слова :', ans_6)
# write_answer(6, ans_6)

In [75]:

# nword = 2
# for i in np.argpartition(lr.coef_[0], -nword)[-nword:]:
#     print('{:5d} {:.4f} - {}'.format(i, lr.coef_[0][i], cv.get_feature_names()[i]))

15246 0.4443 - great
14159 0.5561 - fun


In [140]:
df = pd.DataFrame(lr.coef_[0], columns=['cf'])
df['cf_abs'] = abs(df.cf)
ser = df.sort_values(by=['cf_abs'],  ascending=False)
ser.head()

,cf,cf_abs
2954,-0.782177,0.782177
37056,-0.636619,0.636619
39195,-0.592902,0.592902
14159,0.556066,0.556066
38417,-0.508179,0.508179


In [139]:
nword = 20 
for i, cf in zip(ser.head(nword).index, ser.head(nword).cf.values):
    print('{: >8.4f} - {}'.format(cf, cv.get_feature_names()[i]))


 -0.7822 - bad
 -0.6366 - unfortunately
 -0.5929 - worst
  0.5561 - fun
 -0.5082 - waste
 -0.5040 - nothing
 -0.4665 - script
 -0.4652 - awful
 -0.4632 - boring
 -0.4600 - only
  0.4443 - great
 -0.4427 - plot
 -0.4421 - poor
 -0.4294 - reason
  0.4267 - back
 -0.4172 - looks
 -0.3945 - mess
 -0.3928 - supposed
 -0.3854 - lame
  0.3728 - quite


In [141]:
ans_6 = ' '.join([cv.get_feature_names()[i] for i in [2954, 37056]])
print('6. Два самых важным для модели признакам слова :', ans_6)
write_answer(6, ans_6)

6. Два самых важным для модели признакам слова : bad unfortunately
